# Importação de bibliotecas

In [ ]:
from copy import deepcopy
import os
import pickle
import sys
from tqdm import tqdm
import numpy as np
import scipy.io as io
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle, resample


In [ ]:
from pump_utils import process, plots, models

# Manipulação dos dados

## Abrindo o diretório


In [ ]:
path='F:\FACUL\FACULDADE\PG1\Dados'
os.listdir(path)
Df_v1 = pd.read_csv(f'{path}\\sensor.csv')

## Obtendo o Dataframe

In [ ]:
Df_v2 = Df_v1.drop(columns='Unnamed: 0') #Coluna repete index
# Cols=Df_v2.columns

In [ ]:
Df_v2

In [ ]:
Df_v3=Df_v2


# Df_v2[Cols[3]].loc[Df_v2[Cols[3]].isnull()]

# for Title in Cols:
    # for Value in range(len(Df_v2[Title])):
    #     if Df_v2[Title][Value] == 'NaN' :
    #         Df_v2[Title][Value].interpolate(method ='linear', limit_direction ='forward')
    
    # if Df_v2[Title].isnull().sum() > 0 :
    #    ContNull = Df_v2[Title].isnull().sum()
    #     for id in range(ContNull):
    #         Df_v2[Title].loc[Df_v2[Title].isnull()]

In [ ]:
# Df_v2[Cols[1]].loc[Df_v2[Cols[1]].isnull()]
# Df_v2[Cols[1]].iloc[17157]

plt.plot(Df_v3['sensor_13'][:100])


In [ ]:
sns.heatmap(Df_v3.isnull(),cbar=False,cmap='viridis')

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sns.countplot(data=Df_v3, x='machine_status', ax=ax)

In [ ]:
# mudança no formato da coluna "timestamp"
Df_v3['timestamp'] = Df_v3['timestamp'].astype('datetime64')

# visualização temporal dos dados
plots.sensor_plot(Df_v3, sensors=[f'sensor_{i}' for i in ['00','02', '10', '18']])

# 

In [ ]:
Df_v3['b_machine_status'] = Df_v3['machine_status'].apply(lambda x: x if x == 'NORMAL' else 'ANOMALY')

In [ ]:
# verificação das distribuições dos sensores
plots.dist_plot(Df_v3, sensors=[f'sensor_{i}' for i in ['00', '02', '05', '27']])

In [ ]:
plots.boxplot(Df_v3, sensors=[f'sensor_{i}' for i in ['00', '02', '05', '27']])

In [ ]:
# Df_v3.plot(subplots =True, sharex = True, figsize = (20,50))

In [ ]:
plots.corr_plot(Df_v3, th=.7)

Com a correlação feita acima, é possível selecionar os sensores que possuem uma correlação acima de 70% e ignorar os restantes.
Os sensores são: ['00','01','02','04','05','06','07','08','09','10','11','12','13','38','39','40','41','44','46','48','49','50']

In [ ]:
# verificação da correlação multivariada
plots.mvar_corr(Df_v3, th=.75)

Com a correlação das importâncias realizada acima, é possível filtrar novamente as melhores variáveis para o problema.
Os sensores são: ['00', '01', '03,'04','05','06','09','10','11','13','14','19','22','26','28','29','34','36','48','51']

In [ ]:
# verificação das variáveis redundantes
num_sensors = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '10', '11', '12', '13', '38', '40', '45', '48', '49', '50','51']
selected_sensors = [f'sensor_{i}' for i in num_sensors]
plots.redundant_features(Df_v3, sensors=selected_sensors)

In [ ]:
null_list = Df_v3.isnull().sum()
(null_list[null_list != 0] / Df_v3.shape[0])*100

In [ ]:
# exclusão dos sensores 0, 15 e 50
pdata = Df_v3[selected_sensors].copy()
pdata['b_machine_status'] = Df_v3['b_machine_status'].copy()
pdata.drop(columns=['sensor_00', 'sensor_50'], inplace=True)

# interpolação dos demais sensores
pdata.interpolate(axis=0, inplace=True)

In [ ]:
Df_v3['timestamp']

In [ ]:
# Df_v4=Df_v3
# for i in range(16,26):
#     Df_v4=Df_v4.drop(columns=f'sensor_{i}') 
# for i in range(38,41):
#     Df_v4=Df_v4.drop(columns=f'sensor_{i}')
# for i in range(46,49):
#     Df_v4=Df_v4.drop(columns=f'sensor_{i}')


In [ ]:
Cols=Df_v3.columns
# Df_v3['sensor_00'][0:10].mean()
x=0
# for n in Cols:
#     size = len(Df_v3[n])
#     for m in range(9,size,10):
#         test[n] = Df_v3[n][x:m].mean()
#         x=x+m

In [ ]:
# Df_v4.plot(subplots =True, sharex = True, figsize = (20,50))

In [ ]:
# for m in range(len(Df_v3['machine_status'])):
#     if Df_v3['machine_status'][m] == 'NORMAL':
#         Df_v3 = 0
#     if Df_v3['machine_status'][m] == 'BROKEN':
#         Df_v3 = 1
#     if Df_v3['machine_status'][m] == 'RECOVERING':
#         Df_v3 = 2
    


In [ ]:
Df_v3

In [ ]:
# Cols=Df_v3.columns
# # Df_v3['sensor_00'][0:10].mean()
# test={}
# x=0
# cont=0
# for n in Cols:
#     if n == 'timestamp':
#         pass
#     else:
#         size = len(Df_v3[n])
#         for m in range(9,size,10):
#             test[cont] = {f'{n}',Df_v3[n][x:m].mean()}
#             cont=cont+1
#             x=m+1

In [ ]:
# test
Cols
# x
# n == 'timestamp'

## Transformando em .Parquet

In [ ]:
# os.makedirs('accel_data/', exist_ok=True)

# prefix = path.split('/')[-1] #Prefixo do nome a ser salvo no arquivo. EX. "LRV4306_acc.parquet"
Df_v3.to_parquet('F:\FACUL\FACULDADE\PG1\Dados\Dados_Pump.parquet', index=False)

## Separando os sensores

In [ ]:
Df_bearingTemp=pd.DataFrame()
Df_vibration=pd.DataFrame()
Df_motorProprieties=pd.DataFrame()
Df_pumpSpeed=pd.DataFrame()
Df_pumpFlow=pd.DataFrame()
Df_pumpLube=pd.DataFrame()
Df_pumpPressure=pd.DataFrame()


In [ ]:
# Df_v2.unique()
k=0
# Df_v2.loc[Df_v2['sensor_00']]
# Df_v2.isna().describe()
Df_final = pd.read_parquet(path+r'\Dados_Pump.parquet')
for column in tqdm(Cols):
    if column == 'timestamp' or column == 'machine_status':
        pass
    else:
        name_var = column.split('_')
        name_var = name_var[1]
        # try:
        if name_var >= '01' and name_var <= '12' or name_var >= '14' and name_var <= '17':
            Df_motorProprieties[f'sensor_{name_var}'] = Df_final[f'sensor_{name_var}']

        if name_var == '00' or name_var == '13' or name_var == '18':
            Df_vibration[f'sensor_{name_var}'] = Df_final[f'sensor_{name_var}']

        if name_var >= '19' and name_var <= '33':
            Df_pumpSpeed[f'sensor_{name_var}'] = Df_final[f'sensor_{name_var}']

        if name_var >= '34' and name_var <= '36':
            Df_pumpFlow[f'sensor_{name_var}'] = Df_final[f'sensor_{name_var}']

        if name_var >= '37' and name_var <= '39':
            Df_pumpLube[f'sensor_{name_var}'] = Df_final[f'sensor_{name_var}']

        if name_var >= '40' and name_var <= '47':
            Df_bearingTemp[f'sensor_{name_var}'] = Df_final[f'sensor_{name_var}']

        if name_var >= '48' and name_var <= '51':
            Df_pumpPressure[f'sensor_{name_var}'] = Df_final[f'sensor_{name_var}']

In [ ]:

Df_motorProprieties.rename({'sensor_01':'Motor Frequency A','sensor_02':'Motor Frequency B','sensor_03':'Motor Frequency C',
'sensor_04':'Motor Speed','sensor_05':'Motor Current','sensor_06':'Motor Active Power','sensor_07':'Motor Apparent Power','sensor_08':'Motor Reactive Power',
'sensor_09':'Motor Shaft Power','sensor_10':'Motor Phase Current A','sensor_11':'Motor Phase Current B','sensor_12':'Motor Phase Current C','sensor_14':'Motor Phase Voltage AB',
'sensor_16':'Motor Phase Voltage BC','sensor_17':'Motor Phase Voltage CA'},axis=1,inplace=True)

Df_motorProprieties.head()

# SENSOR_01 - Motor Frequency A
# SENSOR_02 - Motor Frequency B
# SENSOR_03 - Motor Frequency C
# SENSOR_04 - Motor Speed
# SENSOR_05 - Motor Current
# SENSOR_06 - Motor Active Power
# SENSOR_07 - Motor Apparent Power
# SENSOR_08 - Motor Reactive Power
# SENSOR_09 - Motor Shaft Power  
# SENSOR_10 - Motor Phase Current A
# SENSOR_11 - Motor Phase Current B
# SENSOR_12 - Motor Phase Current C
# SENSOR_14 - Motor Phase Voltage AB
# SENSOR_16 - Motor Phase Voltage BC
# SENSOR_17 - Motor Phase Voltage CA

In [ ]:
Df_vibration.rename({'sensor_00': 'Motor Casing Vibration', 'sensor_13':'Motor Coupling Vibration', 'sensor_18':'Pump Casing Vibration'},axis=1,inplace=True)

Df_vibration

# SENSOR_00 - Motor Casing Vibration
# SENSOR_13 - Motor Coupling Vibration
# SENSOR_18 - Pump Casing Vibration

In [ ]:
Df_pumpSpeed.rename({'sensor_19': 'Pump Stage 1 Impeller Speed', 'sensor_20':'Pump Stage 1 Impeller Speed', 'sensor_21':'Pump Stage 1 Impeller Speed', 'sensor_22':'Pump Stage 1 Impeller Speed',
'sensor_23':'Pump Stage 1 Impeller Speed', 'sensor_24':'Pump Stage 1 Impeller Speed', 'sensor_25':'Pump Stage 2 Impeller Speed', 'sensor_26':'Pump Stage 2 Impeller Speed', 
'sensor_27':'Pump Stage 2 Impeller Speed', 'sensor_28':'Pump Stage 2 Impeller Speed', 'sensor_29':'Pump Stage 2 Impeller Speed', 'sensor_30':'Pump Stage 2 Impeller Speed',
'sensor_31':'Pump Stage 2 Impeller Speed', 'sensor_32':'Pump Stage 2 Impeller Speed', 'sensor_33':'Pump Stage 2 Impeller Speed'},axis=1,inplace=True)

Df_pumpSpeed.head()

# SENSOR_19 - Pump Stage 1 Impeller Speed
# SENSOR_20 - Pump Stage 1 Impeller Speed
# SENSOR_21 - Pump Stage 1 Impeller Speed
# SENSOR_22 - Pump Stage 1 Impeller Speed
# SENSOR_23 - Pump Stage 1 Impeller Speed
# SENSOR_24 - Pump Stage 1 Impeller Speed
# SENSOR_25 - Pump Stage 2 Impeller Speed
# SENSOR_26 - Pump Stage 2 Impeller Speed
# SENSOR_27 - Pump Stage 2 Impeller Speed
# SENSOR_28 - Pump Stage 2 Impeller Speed
# SENSOR_29 - Pump Stage 2 Impeller Speed
# SENSOR_30 - Pump Stage 2 Impeller Speed
# SENSOR_31 - Pump Stage 2 Impeller Speed
# SENSOR_32 - Pump Stage 2 Impeller Speed
# SENSOR_33 - Pump Stage 2 Impeller Speed

In [ ]:
Df_pumpFlow.rename({'sensor_34': 'Pump Inlet Flow', 'sensor_35':'Pump Discharge Flow', 'sensor_36':'Pump UNKNOWN'},axis=1,inplace=True)
Df_pumpFlow

# SENSOR_34 - Pump Inlet Flow
# SENSOR_35 - Pump Discharge Flow
# SENSOR_36 - Pump UNKNOWN

In [ ]:
Df_bearingTemp.rename({'sensor_40': 'Pump Thrust Bearing Active Temp', 'sensor_41':'Motor Non Drive End Radial Bearing Temp 1', 'sensor_42':'Motor Non Drive End Radial Bearing Temp 2',
'sensor_43':'Pump Thrust Bearing Inactive Temp','sensor_44':'Pump Drive End Radial Bearing Temp 1','sensor_45':'Pump non Drive End Radial Bearing Temp 1',
'sensor_46':'Pump non Drive End Radial Bearing Temp 2','sensor_47':'Pump Drive End Radial Bearing Temp 2'},axis=1,inplace=True)

Df_bearingTemp

# SENSOR_40 - Pump Thrust Bearing Active Temp
# SENSOR_41 - Motor Non Drive End Radial Bearing Temp 1
# SENSOR_42 - Motor Non Drive End Radial Bearing Temp 2
# SENSOR_43 - Pump Thrust Bearing Inactive Temp
# SENSOR_44 - Pump Drive End Radial Bearing Temp 1
# SENSOR_45 - Pump non Drive End Radial Bearing Temp 1
# SENSOR_46 - Pump Non Drive End Radial Bearing Temp 2
# SENSOR_47 - Pump Drive End Radial Bearing Temp 2

In [ ]:
Df_pumpPressure.rename({'sensor_48': 'Pump Inlet Pressure', 'sensor_49':'Pump Temp Unknown', 'sensor_50':'Pump Discharge Pressure 1', 'sensor_51':'Pump Discharge Pressure 2'},axis=1,inplace=True)

Df_pumpPressure

# SENSOR_48 - Pump Inlet Pressure
# SENSOR_49 - Pump Temp Unknown
# SENSOR_50 - Pump Discharge Pressure 1
# SENSOR_51 - Pump Discharge Pressure 2

In [ ]:
# SENSOR_00 - Motor Casing Vibration
# SENSOR_13 - Motor Coupling Vibration
# SENSOR_18 - Pump Casing Vibration

# SENSOR_01 - Motor Frequency A
# SENSOR_02 - Motor Frequency B
# SENSOR_03 - Motor Frequency C
# SENSOR_04 - Motor Speed
# SENSOR_05 - Motor Current
# SENSOR_06 - Motor Active Power
# SENSOR_07 - Motor Apparent Power
# SENSOR_08 - Motor Reactive Power
# SENSOR_09 - Motor Shaft Power  
# SENSOR_10 - Motor Phase Current A
# SENSOR_11 - Motor Phase Current B
# SENSOR_12 - Motor Phase Current C
# SENSOR_14 - Motor Phase Voltage AB
# SENSOR_16 - Motor Phase Voltage BC
# SENSOR_17 - Motor Phase Voltage CA



# SENSOR_19 - Pump Stage 1 Impeller Speed
# SENSOR_20 - Pump Stage 1 Impeller Speed
# SENSOR_21 - Pump Stage 1 Impeller Speed
# SENSOR_22 - Pump Stage 1 Impeller Speed
# SENSOR_23 - Pump Stage 1 Impeller Speed
# SENSOR_24 - Pump Stage 1 Impeller Speed
# SENSOR_25 - Pump Stage 2 Impeller Speed
# SENSOR_26 - Pump Stage 2 Impeller Speed
# SENSOR_27 - Pump Stage 2 Impeller Speed
# SENSOR_28 - Pump Stage 2 Impeller Speed
# SENSOR_29 - Pump Stage 2 Impeller Speed
# SENSOR_30 - Pump Stage 2 Impeller Speed
# SENSOR_31 - Pump Stage 2 Impeller Speed
# SENSOR_32 - Pump Stage 2 Impeller Speed
# SENSOR_33 - Pump Stage 2 Impeller Speed

# SENSOR_34 - Pump Inlet Flow
# SENSOR_35 - Pump Discharge Flow
# SENSOR_36 - Pump UNKNOWN

# SENSOR_37 - Pump Lube Oil Overhead Reservoir Level
# SENSOR_38 - Pump Lube Oil Return Temp
# SENSOR_39 - Pump Lube Oil Supply Temp

# SENSOR_40 - Pump Thrust Bearing Active Temp
# SENSOR_41 - Motor Non Drive End Radial Bearing Temp 1
# SENSOR_42 - Motor Non Drive End Radial Bearing Temp 2
# SENSOR_43 - Pump Thrust Bearing Inactive Temp
# SENSOR_44 - Pump Drive End Radial Bearing Temp 1
# SENSOR_45 - Pump non Drive End Radial Bearing Temp 1
# SENSOR_46 - Pump Non Drive End Radial Bearing Temp 2
# SENSOR_47 - Pump Drive End Radial Bearing Temp 2

# SENSOR_48 - Pump Inlet Pressure
# SENSOR_49 - Pump Temp Unknown
# SENSOR_50 - Pump Discharge Pressure 1
# SENSOR_51 - Pump Discharge Pressure 2
# Pump Status

## Correlação

In [ ]:
# rs = np.random.RandomState(0)
# df = pd.DataFrame(rs.rand(10, 10))
corr = Df_v3.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)
# 'RdBu_r', 'BrBG_r', & PuOr_r are other good diverging colormaps

In [ ]:
fig, axs = plt.subplots(2,figsize=(30,15))
# corr2=Df_motorProprieties.corr()
# corr2.style.background_gradient(cmap='coolwarm').set_precision(2)
# sns.heatmap(corr2, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
#             square=True, ax=ax)
x=range(0,220320,1)
axs[0].plot(x, Df_pumpFlow[Df_pumpFlow.columns[1]])
axs[1].plot(x, Df_bearingTemp[Df_bearingTemp.columns[1]])